In [5]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.callbacks import CSVLogger
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
# Provide the correct file path for PeterMac data
data_file_path = Path('../Resources/PeterMac_HRD_Validation.csv')
data_df = pd.read_csv(data_file_path)

In [7]:
#  YOUR CODE GOES HERE
columns_to_drop = ["SampleID", "SeqRunID", "DDMSampleID","MonthsOld"]

# Drop the specified columns from the DataFrame
data_df = data_df.drop(columns=columns_to_drop, axis=1)
data_df.head()

,Run,Source,Purity,MIDS,TotalReads(M),lpWGSReads(M),TargetPanelReads(M),%ReadslpWGS,%ReadsPanel,1000x,...,ResNoise,SignalNoiseRatio,QAStatus,Gene,Variant,%VariantFraction,MyriadGIScore,MyriadGIStatus,SOPHiAGIIndex,SophiaGIStatus
0,1,AZ,20,2,7.3,5.9,1.4,81%,19%,1%,...,0.13,2.95,Medium,.,.,.,51,1,3.2,1
1,1,AZ,30,3,7.3,5.6,1.7,76%,24%,2%,...,0.11,2.91,High,.,.,.,20,2,-15.7,2
2,1,AZ,20,4,9.6,6.1,3.5,64%,36%,41%,...,0.1,1.64,High,.,.,.,17,2,-4.6,2
3,1,AZ,20,6,8.9,5.6,3.3,63%,37%,16%,...,0.09,3.49,High,.,.,.,29,2,-4.6,2
4,1,AZ,60,7,8.6,5.0,3.6,58%,42%,2%,...,0.11,2.18,High,.,.,.,29,2,-8.2,2


In [8]:
# Convert columns we will not bin, from pobject type to numeric
column_names_to_convert = ['Purity']

# Step 1: Check the current data types of the columns
print(data_df[column_names_to_convert].dtypes)

# Step 2: Convert each column to numeric (if possible)
for col in column_names_to_convert:
    data_df[col] = pd.to_numeric(data_df[col], errors='coerce')

# Step 3: Check the new data types of the columns after the conversion
print(data_df[column_names_to_convert].dtypes)
# print(data_df["Source"].dtypes)

Purity    object
dtype: object
Purity    float64
dtype: object


In [9]:
data_df['PurityPloidyRatio'] = data_df['PurityPloidyRatio'].replace('-', 0.0)
data_df['ResNoise'] = data_df['ResNoise'].replace('-', 0.0)
data_df['SignalNoiseRatio'] = data_df['SignalNoiseRatio'].replace('-', 0.0)

data_df['Gene'] = data_df['Gene'].replace('.', 'Unlisted')


# data_df['MonthsOld'] = data_df['MonthsOld'].fillna(0.0)
# data_df['MonthsOld'] = data_df['MonthsOld'].replace('.', 0.0)
data_df['Purity'] = data_df['Purity'].replace('.', 0.0)
data_df['%VariantFraction'] = data_df['%VariantFraction'].replace('.', 0.0)

# Convert the 'Purity' column to numeric, replacing '.' with 0.0
data_df['Purity'] = pd.to_numeric(data_df['Purity'], errors='coerce').fillna(0.0)

data_df['DupFrac'] = data_df['DupFrac'].replace('%', '', regex=True).astype(float)
data_df['%ReadslpWGS'] = data_df['%ReadslpWGS'].replace('%', '', regex=True).astype(float)
data_df['%ReadsPanel'] = data_df['%ReadsPanel'].replace('%', '', regex=True).astype(float)

data_df['Variant'] = data_df['Variant'].replace('.', 'Unlisted')

# Apply label encoding to 'Variant' column
label_encoder = LabelEncoder()
data_df['Variant'] = label_encoder.fit_transform(data_df['Variant'].astype(str))

data_df['1000x'] = data_df['1000x'].replace('%', '', regex=True).astype(float)

data_df['500x'] = data_df['1000x'].replace('%', '', regex=True).astype(float)
data_df['200x'] = data_df['1000x'].replace('%', '', regex=True).astype(float)
data_df['100x'] = data_df['1000x'].replace('%', '', regex=True).astype(float)
data_df['50x'] = data_df['1000x'].replace('%', '', regex=True).astype(float)
data_df['25x'] = data_df['1000x'].replace('%', '', regex=True).astype(float)

In [10]:
def calculate_non_agreement(row):
    if row['MyriadGIStatus'] == 1 and row['SophiaGIStatus'] == 1:
        return 1
    elif row['MyriadGIStatus'] == 1 and row['SophiaGIStatus'] == 2:
        return 0
    elif row['MyriadGIStatus'] == 1 and row['SophiaGIStatus'] == 3:
        return 0
    elif row['MyriadGIStatus'] == 1 and row['SophiaGIStatus'] == 4:
        return 0
    elif row['MyriadGIStatus'] == 2 and row['SophiaGIStatus'] == 1:
        return 0
    elif row['MyriadGIStatus'] == 2 and row['SophiaGIStatus'] == 2:
        return 1
    elif row['MyriadGIStatus'] == 2 and row['SophiaGIStatus'] == 3:
        return 0
    elif row['MyriadGIStatus'] == 2 and row['SophiaGIStatus'] == 4:
        return 0
    else:
        return None

# Apply the function to create the 'Non-agreement' column
data_df['Non-agreement'] = data_df.apply(calculate_non_agreement, axis=1)

In [11]:
# Create one-hot columns for these 3 columns
onehot_cols = ["QAStatus", "Gene", "Variant"]  #"PurityPloidyRatio"]

# Use get_dummies() to one-hot encode only the categorical columns
one_hot_encoded = pd.get_dummies(data_df[onehot_cols])

# Concatenate the one-hot encoded columns with the original DataFrame
data_df = pd.concat([data_df, one_hot_encoded], axis=1)

# After this, you can print the data types of columns in the 'data_df' DataFrame
column_types = data_df.dtypes

In [12]:
# 1  pca method on application_df which has had bucketing performed for the original model.
data_df_x = data_df.copy() 
columns_to_drop = ["Gene", "Non-agreement", "MyriadGIStatus", "SophiaGIStatus", "MyriadGIScore", "SOPHiAGIIndex", "QAStatus"]
data_df_x = data_df_x.drop(columns=columns_to_drop, axis=1)


#data_df_x = data_df_x.drop(columns=["Gene","Source","Non_agreement", "MyriadGIStatus", "SophiaGIStatus", "MyriadGIScore", "SOPHiAGIIndex","QAStatus"], axis=1)
data_df_x.columns

Index(['Run', 'Source', 'Purity', 'MIDS', 'TotalReads(M)', 'lpWGSReads(M)',
       'TargetPanelReads(M)', '%ReadslpWGS', '%ReadsPanel', '1000x', '500x',
       '200x', '100x', '50x', '25x', 'DupFrac', 'LowCovRegions',
       'PurityPloidyRatio', 'ResNoise', 'SignalNoiseRatio', 'Variant',
       '%VariantFraction', 'Variant', 'QAStatus_High', 'QAStatus_Low',
       'QAStatus_Medium', 'Gene_BRCA1', 'Gene_BRCA2', 'Gene_RAD51D',
       'Gene_Unlisted'],
      dtype='object')

In [13]:
# Replace "Deleted" with 0 in the '%ReadslpWGS' column
data_df_x['%VariantFraction'] = data_df_x['%VariantFraction'].replace('Deleted', 0)

# Convert the column to numeric (float) format
data_df_x['%VariantFraction'] = pd.to_numeric(data_df_x['%VariantFraction'])

In [14]:
# Check data types of the columns in data_df_x
print(data_df_x.dtypes)

# Check for missing values in data_df_x
print(data_df_x.isnull().sum())

# Convert the '%VariantFraction' column to numeric values, invalid values will be converted to NaN
data_df_x['%VariantFraction'] = pd.to_numeric(data_df_x['%VariantFraction'], errors='coerce')

# Create a mask to identify rows with NaN values in the '%VariantFraction' column
invalid_rows_mask = data_df_x['%VariantFraction'].isna()

# Use the mask to filter the DataFrame and get the rows with invalid values
invalid_rows = data_df_x[invalid_rows_mask]

# Check if any non-numeric values still exist in data_df_x
non_numeric_values = data_df_x.apply(pd.to_numeric, errors='coerce').isnull().sum()
print(non_numeric_values)

Run                      int64
Source                  object
Purity                 float64
MIDS                     int64
TotalReads(M)          float64
lpWGSReads(M)          float64
TargetPanelReads(M)    float64
%ReadslpWGS            float64
%ReadsPanel            float64
1000x                  float64
500x                   float64
200x                   float64
100x                   float64
50x                    float64
25x                    float64
DupFrac                float64
LowCovRegions            int64
PurityPloidyRatio       object
ResNoise                object
SignalNoiseRatio        object
Variant                  int64
%VariantFraction       float64
Variant                  int64
QAStatus_High            uint8
QAStatus_Low             uint8
QAStatus_Medium          uint8
Gene_BRCA1               uint8
Gene_BRCA2               uint8
Gene_RAD51D              uint8
Gene_Unlisted            uint8
dtype: object
Run                    0
Source                 0
Purity

In [15]:
columns_to_drop = ['Variant', 'Source']
data_df_x = data_df_x.drop(columns=columns_to_drop)

data_df_x.columns

Index(['Run', 'Purity', 'MIDS', 'TotalReads(M)', 'lpWGSReads(M)',
       'TargetPanelReads(M)', '%ReadslpWGS', '%ReadsPanel', '1000x', '500x',
       '200x', '100x', '50x', '25x', 'DupFrac', 'LowCovRegions',
       'PurityPloidyRatio', 'ResNoise', 'SignalNoiseRatio', '%VariantFraction',
       'QAStatus_High', 'QAStatus_Low', 'QAStatus_Medium', 'Gene_BRCA1',
       'Gene_BRCA2', 'Gene_RAD51D', 'Gene_Unlisted'],
      dtype='object')

In [16]:
invalid_rows = data_df_x['%VariantFraction'].apply(lambda x: not str(x).replace('.', '').isnumeric())

# Display the rows containing the invalid values
print(data_df_x[invalid_rows])

Empty DataFrame
Columns: [Run, Purity, MIDS, TotalReads(M), lpWGSReads(M), TargetPanelReads(M), %ReadslpWGS, %ReadsPanel, 1000x, 500x, 200x, 100x, 50x, 25x, DupFrac, LowCovRegions, PurityPloidyRatio, ResNoise, SignalNoiseRatio, %VariantFraction, QAStatus_High, QAStatus_Low, QAStatus_Medium, Gene_BRCA1, Gene_BRCA2, Gene_RAD51D, Gene_Unlisted]
Index: []

[0 rows x 27 columns]


In [17]:
# Split our preprocessed data into our features and target arrays
y2 = data_df["Non-agreement"].values
X2 = data_df_x.values

# Split the preprocessed data into a training and testing dataset
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=78)

# Now perform PCA only on the training data
pca = PCA(n_components=3)
X2_train_pca = pca.fit_transform(X2_train)

# Apply the same PCA transformation to the testing data
X2_test_pca = pca.transform(X2_test)

# Check the number of records in the original dataset
print(data_df_x.shape)

# Check the number of records after splitting into training and testing sets
print(X2_train.shape, X2_test.shape)

# Check the number of records after PCA transformation
print(X2_train_pca.shape, X2_test_pca.shape)

# Create a new DataFrame with the PCA data for both training and testing sets
df_train_pca = pd.DataFrame(X2_train_pca, columns=["PC1", "PC2", "PC3"])
df_test_pca = pd.DataFrame(X2_test_pca, columns=["PC1", "PC2", "PC3"])

(139, 27)
(111, 27) (28, 27)
(111, 3) (28, 3)


In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X2_scaler = scaler.fit(X2_train)

# Scale the data
X2_train_scaled = X2_scaler.transform(X2_train)
X2_test_scaled = X2_scaler.transform(X2_test)

In [22]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=2), activation=activation, input_dim=27))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [23]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=100,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [24]:
# Run the kerastuner search for best hyperparameters
tuner.search(X2_train_scaled,y2_train,epochs=100,validation_data=(X2_test_scaled,y2_test))

Trial 500 Complete [00h 00m 07s]
val_accuracy: 0.7142857313156128

Best val_accuracy So Far: 0.8571428656578064
Total elapsed time: 00h 19m 18s
INFO:tensorflow:Oracle triggered exit


In [25]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 11,
 'num_layers': 3,
 'units_0': 13,
 'units_1': 13,
 'units_2': 13,
 'units_3': 11,
 'units_4': 29,
 'units_5': 25,
 'tuner/epochs': 12,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 3,
 'tuner/round': 1,
 'tuner/trial_id': '0151'}

In [26]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X2_test_scaled,y2_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.5823 - accuracy: 0.8571 - 245ms/epoch - 245ms/step
Loss: 0.58232581615448, Accuracy: 0.8571428656578064
